In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
# tag: remove-cell applied

# Dichotomous Data

## Quickstart

To run a dichotomous dataset:

In [ ]:
import pybmds

dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

# create a BMD session
session = pybmds.Session(dataset=dataset)

# add all default models
session.add_default_models()

# execute the session
session.execute()

# recommend a best-fitting model
session.recommend()

model_index = session.recommender.results.recommended_model_index
if model_index:
    model = session.models[model_index]
    display(model.plot())
    print(model.text())

# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

## Create a dichotomous dataset

To create a dichotomous dataset, you need a list of doses, incidences, and the total number of subjects. As shown above, you use the ```DichotomousDataset()``` function and insert your data. You can specify the title of the dataset and the dose units. You can also create a plot of the dataset which will have the title that you name the dataset and the specified units. For example, here the dataset name is "ChemX Nasal Lesion Incidence".

In [ ]:
dataset = pybmds.DichotomousDataset(
    name="ChemX Nasal Lesion Incidence",
    dose_units="ppm",
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

dataset.plot()

## Fitting one model to dataset

If you want to fit only one model to your dataset, you can fit the specific model to the dataset and print the results such as the BMD, BMDL, BMDU, p-value, AIC, etc. Say you want to fit the Logistic model, you can run the code below and print the results:

In [ ]:
from pybmds.models import dichotomous

model = dichotomous.Logistic(dataset)
model.execute()
model.plot()

You can also print an output report:

In [ ]:
print(model.text())

The individual models that you can fit are shown below. Note that the degrees of the Multistage model can be increased to a maximum of the lesser of n-1 or 8 (as specified in the BMDS User Guide).

In [ ]:
dichotomous.QuantalLinear(dataset)
dichotomous.Multistage(dataset, settings = {"degree": 2})
dichotomous.Multistage(dataset, settings = {"degree": 3})
dichotomous.Logistic(dataset)
dichotomous.LogLogistic(dataset)
dichotomous.Probit(dataset)
dichotomous.LogProbit(dataset)
dichotomous.Gamma(dataset)
dichotomous.Weibull(dataset)
dichotomous.DichotomousHill(dataset)

## Changing initial model parameter settings

If you want to see a preview of the initial parameter settings, you can run:


In [ ]:
model = dichotomous.Logistic(dataset)
print(model.priors_tbl())

You can also change the initial parameter settings shown above for any run of a single dichotomous model. For example, continuing with the Logistic model example, you can change the minimum and maximum range for `a` to be from -10 to 10 and `b` to be from 0 to 50.

In [ ]:
model.settings.priors.update('a', min_value=-10, max_value=10)
model.settings.priors.update('b', min_value=0, max_value=50)
print(model.priors_tbl())

You can change the range and initial value for any parameter in the model by following the same steps above.

## Run all models and select the best fit

As shown in the Quickstart above, you can run all of the default dichotomous models and select the recommended model. To run all the default models, save the results, and save the plot of the fit of the recommended model with the data:

In [ ]:
dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

# create a BMD session
session = pybmds.Session(dataset=dataset)

# add all default models
session.add_default_models()

# execute the session
session.execute()

# recommend a best-fitting model
session.recommend()

#print recommended model and plot recommended model with dataset
model_index = session.recommender.results.recommended_model_index
if model_index:
    model = session.models[model_index]
    display(model.plot())
    print(model.text())


You can also plot all models:

In [ ]:
session.plot()

In [ ]:
session.plot(colorize=False)

You can generate Excel and word reports:

In [ ]:
# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

## Run subset of models and select best fit

You can select a set of models and find the best fit, rather than using all of the default dichotomous models. For example, to model average on the Logistic, Probit, Quantal Linear, and Weibull models, you can:

In [ ]:
session1 = pybmds.Session(dataset=dataset)
session1.add_model(pybmds.Models.Weibull)
session1.add_model(pybmds.Models.Logistic)
session1.add_model(pybmds.Models.Probit)
session1.add_model(pybmds.Models.QuantalLinear)

session1.execute()
session1.recommend()

model_index = session1.recommender.results.recommended_model_index
if model_index:
    model = session1.models[model_index]
    display(model.plot())
    print(model.text())

## Changing the input settings

The default settings for a dichotomous run use a BMR of 10% Extra Risk and a 95% confidence interval. If you fit just one model to your dataset, you can change these settings by:

In [ ]:
model = dichotomous.Logistic(dataset, settings={
    "bmr": 0.15,
    "bmr_type": pybmds.DichotomousRiskType.AddedRisk
})
print(model.settings.tbl())

If you run all the default models and select the best fit, you can change these settings by:

In [ ]:
session.add_default_models(
    settings = {
        "bmr": 0.15,
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk,
        "alpha": 0.1
    }
)

This would run the dichotomous models for a BMR of 15% Added Risk at a 90% confidence interval.